# Dijkstra's shortest path algorithm.

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

from typing import Any

import numpy as np
import pandas as pd
from IPython.core.display_functions import display
from matplotlib import pyplot as plt
from networkx import DiGraph, set_node_attributes
from teaching_optimization.networks import draw_network
from teaching_optimization.networks.shortest_path_algorithm import (
    ShortestPathAlgorithm,
)


In this lab, you will implement and apply **Dijkstra’s shortest path algorithm** to find the
minimum-cost paths from an origin to all other nodes in a directed network with **nonnegative arc costs**.
You will maintain **labels** (current best distances) and **predecessors**, repeatedly select the node
with the smallest tentative label, and **process** its outgoing arcs to update neighbors' labels.
Along the way, you will learn how to inspect graph data (nodes, arcs, attributes), reconstruct the
**shortest path tree**, and diagnose when Dijkstra is **not applicable** (e.g., negative costs).
The goal is to connect the algorithmic steps (selection, relaxation, termination) with clear network
intuition (paths and costs).

# Some useful functions

Before implementing the algorithm, we investigate some useful functions of the network representation
and illustrate them on an example.

We define a first network example.

In [ ]:
positions = {
    'a': (2, 0),
    'b': (4, -1),
    'c': (4, 1),
    'd': (8, 3),
    'e': (8, 1),
    'f': (8, -1),
    'g': (8, -3),
    'h': (11, -2),
    'i': (12, 2),
}


nodes = list(positions.keys())

arcs = [
    ('a', 'c', 4),
    ('a', 'b', 3),
    ('c', 'd', 8),
    ('c', 'e', 7),
    ('b', 'c', 0.5),
    ('b', 'f', 3.5),
    ('b', 'g', 6),
    ('e', 'd', 0.5),
    ('d', 'h', 4.5),
    ('d', 'i', 10),
    ('e', 'f', 4.5),
    ('f', 'h', 11),
    ('e', 'i', 2.5),
    ('g', 'i', 5.5),
    ('g', 'h', 7.5),
    ('i', 'h', 2),
]


first_network: DiGraph = DiGraph()
for node in nodes:
    first_network.add_node(node, pos=positions[node])
first_network.add_weighted_edges_from(arcs, weight='cost')
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=first_network, attr_edge_labels='cost', ax=ax)
plt.show()


Here is how to obtain the list of arcs and associated data (here, the cost).

In [ ]:
first_arcs = first_network.edges(data=True)
for arc in first_arcs:
    print(arc)


It means that, in order to access the information about an arc, you need to perform the
following statements.

In [ ]:

for arc in first_arcs:
    # For the upstream node
    upstream_node_arc = arc[0]
    # For the downstream node
    downstream_node_arc = arc[1]
    # For the cost
    cost_arc = arc[2]['cost']
    print(f'Cost {upstream_node_arc} -> {downstream_node_arc} = {cost_arc}')


Here is how to obtain the list of nodes and associated data (here, the position).

In [ ]:
first_nodes = first_network.nodes(data=True)
for node in first_nodes:
    print(node)


The data is accessed as follows.

In [ ]:
for node in first_nodes:
    node_name = node[0]
    node_position = node[1]['pos']
    print(f'Coordinates of {node_name}: {node_position}')


Given a node, we can obtain the list of outgoing arcs.

From node 'a'

In [ ]:
outgoing_arcs_from_a = first_network.out_edges('a', data=True)
for arc in outgoing_arcs_from_a:
    print(arc)


From node 'd'

In [ ]:
outgoing_arcs_from_d = first_network.out_edges('d', data=True)
for arc in outgoing_arcs_from_d:
    print(arc)



# Dijkstra's shortest path algorithm

Now we implement Dijkstra's shortest path algorithm. Replace the `...`

In [ ]:


def dijkstra_algorithm(
    the_network: DiGraph, the_cost: str, the_origin: Any
) -> tuple[dict[Any:float] | None, dict[Any, Any | None] | None, pd.DataFrame]:
    """

    :param the_network: network representation
    :param the_cost: name of the cost parameter
    :param the_origin: node at the origin.
    :return:a dict associated each node with their optimal label (or None if problem is unbounded), a dict associating
        each node with its predecessor in the shortest path (or None if the problem is unbounded)
        and a data frame describing the iterations
    """

    # Initialize the labels and the predecessors
    labels = {name: np.inf for name in the_network.nodes}
    labels[the_origin] = 0
    predecessors = {name: None for name in the_network.nodes}
    the_arcs = the_network.edges(data=True)
    # Identify the lowest cost in order to establish a lower bound on the labels
    arc_with_lowest_cost = min(the_arcs, key=lambda x: x[2][the_cost])
    lowest_cost = arc_with_lowest_cost[2][the_cost]
    if lowest_cost < 0:
        error_msg = (
            f"Dijkstra's algorithm cannot be used when some arcs have a negative cost. Here, arc "
            f"{arc_with_lowest_cost} has cost {lowest_cost}"
        )
        raise ValueError(error_msg)

    # Initialize the set of nodes to be treated
    nodes_to_be_treated = {the_origin}

    iteration_number = 0

    reporting_iteration: list[dict[str:Any]] = list()

    # Loop until the set of nodes is empty.
    while nodes_to_be_treated:
        row = {
            'Iteration': iteration_number,
            'Set': str(nodes_to_be_treated),
        }
        # Select the node to be treated during this iteration, and remove it from the set.
        # The node to be treated corresponds to the minimum label in the set.
        current_node = min(
            nodes_to_be_treated, key=lambda a_node: labels[a_node]
        )
        nodes_to_be_treated.remove(current_node)
        row['Node'] = current_node
        for node, label in labels.items():
            row[node] = label
        reporting_iteration.append(row)
        # Consider the list of outgoing arc
        outgoing_arcs = the_network.out_edges(
            current_node, data=True
        )
        for arc in outgoing_arcs:
            upstream_node = arc[0]
            downstream_node = arc[1]
            cost = arc[2][the_cost]
            # Check if the label must be updated
            if (
                labels[downstream_node] > labels[upstream_node] + cost
            ):
                # Update the label
                labels[downstream_node] = (
                    labels[upstream_node] + cost
                )
                # Update the predecessor
                predecessors[
                    downstream_node
                ] = upstream_node
                # Update the set of nodes to be treated
                nodes_to_be_treated.add(
                    downstream_node
                )
        iteration_number += 1

    row = {'Iteration': iteration_number, 'Set': '{}', 'Node': ''}
    for node, label in labels.items():
        row[node] = label
    reporting_iteration.append(row)
    return labels, predecessors, pd.DataFrame(reporting_iteration)



# First example
We run the algorithm on the example above.

In [ ]:
optimal_labels, predecessors, iterations = dijkstra_algorithm(
    the_network=first_network, the_cost='cost', the_origin='a'
)


Optimal labels

In [ ]:
display(optimal_labels)


Predecessors

In [ ]:
display(predecessors)


We add the optimal labels as attributes of the nodes

In [ ]:
set_node_attributes(first_network, optimal_labels, 'label')


Description of the iterations

In [ ]:
display(iterations)



We now write a recursive function to identify the shortest paths

In [ ]:
def shortest_path(node: Any, the_predecessors: dict[Any, Any | None]) -> str:
    """Print the shortest path to a given node, recursively"""

    # First case: if there is  no predecessor, it means that the path is the node itself.
    if the_predecessors[node] is None:
        return str(node)

    # If there is a predecessor, with merge the path to the predecessor with the current node.
    return f'{shortest_path(node=the_predecessors[node], the_predecessors=the_predecessors)} -> {str(node)}'



Print the shortest paths

In [ ]:
for node in first_network.nodes:
    print(shortest_path(node=node, the_predecessors=predecessors))


We create and plot the shortest path graph

In [ ]:
shortest_path_arcs = [
    (upstream, downstream)
    for downstream, upstream in predecessors.items()
    if upstream is not None
]

shortest_path_tree: DiGraph = DiGraph()
for node in nodes:
    shortest_path_tree.add_node(node, pos=positions[node])
shortest_path_tree.add_edges_from(shortest_path_arcs)
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=shortest_path_tree, ax=ax)
plt.show()



# Second example

In [ ]:
positions = {
    'a': (0, 0),
    'b': (2, 1.5),
    'c': (2, -1.5),
    'd': (4, 1.5),
    'e': (4, -1.5),
    'f': (6, 0),
    'g': (8, 1.5),
    'h': (10, 0),
    'i': (8, -1.5),
}


nodes = list(positions.keys())

arcs = [
    ('a', 'b', 8),
    ('a', 'c', 14),
    ('b', 'd', 9),
    ('b', 'e', 7),
    ('b', 'c', 4),
    ('c', 'e', 11),
    ('d', 'e', 12),
    ('d', 'f', 17),
    ('e', 'f', 3),
    ('e', 'i', 8),
    ('f', 'h', 9),
    ('f', 'g', 5),
    ('g', 'h', 3),
    ('i', 'h', 9),
]



We plot the network

In [ ]:
second_network: DiGraph = DiGraph()
for node in nodes:
    second_network.add_node(node, pos=positions[node])
second_network.add_weighted_edges_from(arcs, weight='cost')
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=second_network, attr_edge_labels='cost', ax=ax)
plt.show()


In [ ]:
optimal_labels, predecessors, iterations = dijkstra_algorithm(
    the_network=second_network, the_cost='cost', the_origin='a'
)


Optimal labels

In [ ]:
display(optimal_labels)


Predecessors

In [ ]:
display(predecessors)


We add the optimal labels as attributes of the nodes

In [ ]:
if optimal_labels is not None:
    set_node_attributes(second_network, optimal_labels, 'label')


Description of the iterations

In [ ]:
display(iterations)


Print the shortest paths

In [ ]:
for node in second_network.nodes:
    print(shortest_path(node=node, the_predecessors=predecessors))


We create and plot the shortest path graph

In [ ]:
shortest_path_arcs = [
    (upstream, downstream)
    for downstream, upstream in predecessors.items()
    if upstream is not None
]

shortest_path_tree: DiGraph = DiGraph()
for node in nodes:
    shortest_path_tree.add_node(node, pos=positions[node])
shortest_path_tree.add_edges_from(shortest_path_arcs)
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=shortest_path_tree, ax=ax)
plt.show()


For future exercises, it is possible to perform the same tasks using the package.

Initialization

In [ ]:
the_algorithm = ShortestPathAlgorithm(
    the_network=second_network, the_cost_name='cost', the_origin='a'
)


Running the algorithm

In [ ]:
the_algorithm.shortest_path_algorithm()


Printing the shortest paths

In [ ]:
the_shortest_paths = the_algorithm.list_of_shortest_paths()


Plotting the results

In [ ]:
the_algorithm.plot_shortest_path_tree()


# Third example.

In [ ]:
positions = {
    'a': (0, 0),
    'b': (2, 1.5),
    'c': (2, -1.5),
    'd': (4, 1.5),
    'e': (4, -1.5),
    'f': (6, 0),
    'g': (8, 1.5),
    'h': (10, 0),
    'i': (8, -1.5),
}


nodes = list(positions.keys())

arcs = [
    ('a', 'b', 8),
    ('a', 'c', 14),
    ('b', 'd', 9),
    ('b', 'e', 7),
    ('b', 'c', 4),
    ('c', 'e', -11),
    ('d', 'e', 12),
    ('d', 'f', 17),
    ('e', 'f', 3),
    ('e', 'i', 8),
    ('f', 'h', 9),
    ('f', 'g', 5),
    ('g', 'h', 3),
    ('i', 'h', 9),
]


We plot the network

In [ ]:
third_network: DiGraph = DiGraph()
for node in nodes:
    third_network.add_node(node, pos=positions[node])
third_network.add_weighted_edges_from(arcs, weight='cost')
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=third_network, attr_edge_labels='cost', ax=ax)
plt.show()


We try to run the algorithm

In [ ]:
try:
    optimal_labels, predecessors, iterations = dijkstra_algorithm(
        the_network=third_network, the_cost='cost', the_origin='a'
    )
except ValueError as e:
    print('Algorithm failed.', e)